In [ ]:
%run "0_Workspace_setup.ipynb"

from folium.features import DivIcon
from NHM_helpers.NHM_hydrofabric import *  # give list of names, not stars
from NHM_helpers.map_template import *

In [ ]:
# Create dataframes for hydrofabric elements
hru_gdf, hru_txt, hru_cal_level_txt = create_hru_gdf(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

seg_gdf, seg_txt = create_segment_gdf(
    model_dir,
    GIS_format,
    param_filename,
)

nwis_gages_aoi = fetch_nwis_gage_info(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    nwis_gages_aoi,
    gages_file,
)

default_gages_file = create_default_gages_file(
    model_dir,
    nwis_gages_aoi,
    poi_df,
)

gages_df, gages_txt, gages_txt_nb2 = read_gages_file(
    model_dir,
    poi_df,
    nwis_gages_file,
    gages_file,
)

HW_basins_gdf, HW_basins = make_HW_cal_level_files(hru_gdf)

con.print(
    f"{workspace_txt}\n",
    f"\n{gages_txt}{seg_txt}{hru_txt}",
    f"\n     {hru_cal_level_txt}\n",
    f"\n{gages_txt_nb2}",
)

## Create interactive HRU map

In [ ]:
# Move out to a function

from IPython.display import display
from folium.plugins import MeasureControl
from folium.utilities import Element
from folium.plugins import FloatImage
import base64
import webbrowser

pfile_lat, pfile_lon, zoom, cluster_zoom = folium_map_elements(hru_gdf, poi_df, "")
USGSHydroCached_layer, USGStopo_layer, Esri_WorldImagery, OpenTopoMap = (
    folium_map_tiles()
)
minimap = create_minimap()

################################################
# Create and add hru map
# hru_gdf_map = HW_basins_gdf.to_crs(crs)

hru_map = create_hru_map(hru_gdf)
seg_map_show = create_segment_map_show(seg_gdf)

poi_marker_cluster, poi_marker_cluster_label = create_poi_marker_cluster(
    poi_df, cluster_zoom
)

non_poi_marker_cluster, non_poi_marker_cluster_label = create_non_poi_marker_cluster(
    poi_df, nwis_gages_aoi, gages_df, cluster_zoom
)

m2 = folium.Map()
m2 = folium.Map(
    location=[pfile_lat, pfile_lon],
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    width="100%",
    height="100%",
    control_scale=True,
)

USGStopo_layer.add_to(m2)
OpenTopoMap.add_to(m2)
Esri_WorldImagery.add_to(m2)

# Add widgets
m2.add_child(minimap)
m2.add_child(MeasureControl(position="bottomright"))

hru_cal_map = folium.GeoJson(
    HW_basins_gdf,  # hru_gdf_map,
    style_function=cal_style_function,
    # highlight_function = highlight_function_hru_map,
    name="HRU cal level",
    z_index_offset=40002,
).add_to(m2)


hru_map.add_to(m2)
hw_basins_map = folium.GeoJson(
    HW_basins, style_function=hw_basin_style, name="HW basin boundary"
).add_to(m2)
seg_map_show.add_to(m2)

poi_marker_cluster.add_to(m2)
poi_marker_cluster_label.add_to(m2)

non_poi_marker_cluster.add_to(m2)
non_poi_marker_cluster_label.add_to(m2)

plugins.Fullscreen(position="bottomleft").add_to(m2)
folium.LayerControl(collapsed=True, position="bottomright").add_to(m2)


##add Non-poi gage markers and labels using row df.interowss loop
gages_list = gages_df.index.to_list()
additional_gages = list(set(gages_list) - set(poi_df.poi_id))

# display(m2)
explan_txt = f"nhru (brown): {pdb.dimensions.get('nhru').meta['size']}, nsegment (blue): {pdb.dimensions.get('nsegment').meta['size']},<br>npoigages (black): {pdb.dimensions.get('npoigages').meta['size']}, Additional gages in domain (gray): {len(additional_gages)}"
title_html = f"<h1 style='position:absolute;z-index:100000;font-size: 28px;left:26vw;text-shadow: 3px  3px  3px white,-3px -3px  3px white,3px -3px  3px white,-3px  3px  3px white; '><strong>The {subbasin} Model Hydrofabric</strong><br><h1 style='position:absolute;z-index:100000;font-size: 20px;left:31vw;right:5vw; top:4vw;text-shadow: 3px  3px  3px white,-3px -3px  3px white,3px -3px  3px white,-3px  3px  3px white; '> {explan_txt}</h1>"
# title_html = f"<h1 align='center' style='font-size: 14px;'><strong>NHM simulated {sel_year} {output_var_sel}, {var_units}</strong><br><h1 align='center' style='font-size: 14px;'> {var_desc}. {scale_bar_txt}</h1>"
m2.get_root().html.add_child(Element(title_html))

map_file = f"{html_maps_dir}/hydrofabric_map.html"
m2.save(map_file)

_ = webbrowser.open(map_file)